In [1]:
import urllib.request, json 
import pandas as pd
import os
import functools
import time
import sys
import math
import numpy as np
from numpy.linalg import norm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from env import *

In [2]:
pd.set_option('display.max_columns', None)

Loading dataframe of prepared events.

In [3]:
sessions_df = pd.read_json(SESSION_EVENTS_FROM_BAKED_JSON, orient ='records')
sessions_df[:5]

,Session,Page_0,Page_1,Page_2,Page_3,Page_4,Page_5,Page_6,Page_7,Page_8,Page_9,Page_10,Page_11,Page_12,Page_13,Page_14,Page_15,Page_16,Page_17,Page_18,Page_19,Page_20,Page_21,Page_22,Page_23,Page_24,Page_25,Page_26,Page_27,Page_28,Page_29,Page_30,Page_31,Page_32,Page_33,Page_34,Page_35,Page_36,Page_37,Page_38,Page_39,Page_40,Page_41,Page_42,Page_43,Page_44,Page_45,Page_46,Page_47,Page_48,Page_49,Page_50,Page_51,Page_52,Page_53,Page_54,Page_55,Page_56,Page_57,Page_58,Page_59,Page_60,Page_61,Page_62,Page_63,Page_64,Page_65,Page_66,Page_67
0,S5WLXErzUWNQqkDKnRC1j53PD,"[[[1536, 864, 1536, 722, 5083, None, 164841125...","[[[1536, 864, 1536, 722, 1425, None, 164841125...","[[[1536, 864, 1536, 722, 1577, None, 164841125...","[[[1536, 864, 1536, 722, 1607, None, 164841125...","[[[1536, 864, 1536, 722, 1066, None, 164841126...","[[[1536, 864, 1536, 722, 876, None, 1648411261...","[[[1536, 864, 1536, 722, 948, None, 1648411262...","[[[1536, 864, 1536, 722, 1091, None, 164841126...","[[[1536, 864, 1536, 722, 1037, None, 164841126...","[[[1536, 864, 1536, 722, 1230, None, 164841126...","[[[1536, 864, 1536, 722, 732, None, 1648411266...","[[[1536, 864, 1536, 722, 1087, None, 164841127...","[[[1536, 864, 1536, 722, 1350, None, 164841127...","[[[1536, 864, 1536, 722, 1224, None, 164841127...","[[[1536, 864, 1536, 722, 968, None, 1648411274...","[[[1536, 864, 1536, 722, 957, None, 1648411275...","[[[1536, 864, 1536, 722, 1029, None, 164841127...","[[[1536, 864, 1536, 722, 1155, None, 164841127...","[[[1536, 864, 1536, 722, 1105, None, 164841127...","[[[1536, 864, 1536, 722, 1010, None, 164841127...","[[[1536, 864, 1536, 722, 972, None, 1648411280...","[[[1536, 864, 1536, 722, 1020, None, 164841128...","[[[1536, 864, 1536, 722, 1087, None, 164841128...","[[[1536, 864, 1536, 722, 1290, None, 164841128...","[[[1536, 864, 1536, 722, 1607, None, 164841128...","[[[1536, 864, 1536, 722, 981, None, 1648411287...","[[[1536, 864, 1536, 722, 1005, None, 164841128...","[[[1536, 864, 1536, 722, 1277, None, 164841128...","[[[1536, 864, 1536, 722, 1141, None, 164841129...","[[[1536, 864, 1536, 722, 1008, None, 164841129...","[[[1536, 864, 1536, 722, 1029, None, 164841129...","[[[1536, 864, 1536, 722, 1120, None, 164841129...","[[[1536, 864, 1536, 722, 1189, None, 164841129...","[[[1536, 864, 1536, 722, 1175, None, 164841129...","[[[1536, 864, 1536, 722, 978, None, 1648411297...","[[[1536, 864, 1536, 722, 1030, None, 164841129...","[[[1536, 864, 1536, 722, 1021, None, 164841129...","[[[1536, 864, 1536, 722, 1118, None, 164841130...","[[[1536, 864, 1536, 722, 1305, None, 164841130...","[[[1536, 864, 1536, 722, 1083, None, 164841130...","[[[1536, 864, 1536, 722, 1037, None, 164841130...","[[[1536, 864, 1536, 722, 1049, None, 164841130...","[[[1536, 864, 1536, 722, 1049, None, 164841130...","[[[1536, 864, 1536, 722, 1198, None, 164841130...","[[[1536, 864, 1536, 722, 1048, None, 164841130...","[[[1536, 864, 1536, 722, 1003, None, 164841130...","[[[1536, 864, 1536, 722, 966, None, 1648411310...","[[[1536, 864, 1536, 722, 1045, None, 164841131...","[[[1536, 864, 1536, 722, 1118, None, 164841131...","[[[1536, 864, 1536, 722, 989, None, 1648411313...","[[[1536, 864, 1536, 722, 938, None, 1648411314...","[[[1536, 864, 1536, 722, 1097, None, 164841131...","[[[1536, 864, 1536, 722, 1794, None, 164841131...","[[[1536, 864, 1536, 722, 1068, None, 164841131...","[[[1536, 864, 1536, 722, 972, None, 1648411320...","[[[1536, 864, 1536, 722, 1139, None, 164841132...","[[[1536, 864, 1536, 722, 1207, None, 164841132...","[[[1536, 864, 1536, 722, 1220, None, 164841132...","[[[1536, 864, 1536, 722, 993, None, 1648411324...","[[[1536, 864, 1536, 722, 1119, None, 164841132...","[[[1536, 864, 1536, 722, 1090, None, 164841132...","[[[1536, 864, 1536, 722, 1194, None, 164841132...","[[[1536, 864, 1536, 722, 1284, None, 164841132...","[[[1536, 864, 1536, 722, 1085, None, 164841133...","[[[1536, 864, 1536, 722, 1117, None, 164841133...","[[[1536, 864, 1536, 722, 1139, None, 164841133..

Function for calculation of distance between 2 points.

In [4]:
def distance_calculator(last_coordinates, coordinates):
    return math.sqrt( ((last_coordinates[0] - coordinates[0])**2) + ((last_coordinates[1] - coordinates[1])**2) )

Function for caltulating number of flips on axes.

In [5]:
def flips_counter(coordinates, last_coordinates, x_increase, x_flips, y_increase, y_flips):
    if x_increase == None:
        if coordinates[0] > last_coordinates[0]:
            x_increase = True
        else:
            x_increase = False
    elif x_increase:
        if coordinates[0] < last_coordinates[0]:
            x_increase = False
            x_flips += 1
    else:
        if coordinates[0] > last_coordinates[0]:
            x_increase = True
            x_flips += 1

    if y_increase == None:
        if coordinates[1] > last_coordinates[1]:
            y_increase = True
        else:
            y_increase = False
    elif y_increase:
        if coordinates[1] < last_coordinates[1]:
            y_increase = False
            y_flips += 1
    else:
        if coordinates[1] > last_coordinates[1]:
            y_increase = True
            y_flips += 1
            
    return x_increase, x_flips, y_increase, y_flips

Definition of normaln screen size for normalisation.

In [6]:
normalWidth = 1980
normalHeight = 1080

Function for calculating all metrics. Function is little bit too long and not so clear to understand, but its actually pretty easy. In the cell bellow is function pseudocode.

**Function print_page_stats pseudocode**

    def print_page_stats(session, page, silent = False):

        Define empty list for all metrics

        for wrapper in session[page]:
            for pageview in wrapper:
                Define starting values for pageview related variables

                for event in pageview:
                    if load_info:
                        Load info directly from pageview data

                    if event[2] == 'move':
                        Get initiation time
                        
                        Normalise move coordinates

                        if last_coordinates != [-1, -1]:
                            Calculate distance, x_axis_distance, y_axis_distance, x_flips, y_flips


                    if event[2] == 'click':
                        Normalise click coordinates and save them to list for future use

                    if event[2] == 'scroll':
                        Check if user was scrolling

                Calculate time metrics

                Calculate velocity metrics

                Calculate acceleration metrics

                Find last click on answer button and last click on navigation button

                if last_ans_but_click != None and last_next_but_click != None:
                    Calculate real_ideal_trajectory_diff from first move appearance, to last click on answer button, to last click on navigation button
                    
                    Calculate area under the curve difference of ideal and real trajectory
                    
                    Get reaction time
                    
                    Calculate maximal deviation of movement from ideal trajectory during movement towards last clicked answer button

                Calculate area under the curve as difference of area under the curve of whole movement and area under the curve of ideal trajectory

                if not silent:
                    Print calculated metrics

                Append metrics of pageview to lists (users can visit same pageview multiple times, so every visit is in the list)

        if len(time_arr) != 0:

            Find id of pageview visit with maximal trajectory length
            Return dictionary of coresponding metrics for pageview with maximal trajectory length

        else:
            Return empty dictionary

In [7]:
def print_page_stats(session, page, silent = False):
    time_arr, duration_arr, distance_arr, x_axis_distance_arr, y_axis_distance_arr, real_ideal_trajectory_diff_arr, max_deviation_arr = [], [], [], [], [], [], []
    velocity_arr, x_axis_velocity_arr, y_axis_velocity_arr, acc_arr, x_axis_acc_arr, y_axis_acc_arr, auc_arr = [], [], [], [], [], [], []
    clicks_arr, click_buttons_arr, x_flips_arr, y_flips_arr, scrolling_arr, init_time_arr, react_time_arr = [], [], [], [], [], [], []
    buttons_names = ["Vôbec nesúhlasím", "Nesúhlasím", "Neutrálne", "Súhlasím", "Úplne súhlasím"]
    
    for wrapper in session[page]:
        for pageview in wrapper:
            first_move_at, last_move_at = -1, -1
            last_coordinates = [-1, -1]
            x, y, click_buttons = [], [], []
            x_increase, y_increase = None, None
            x_flips, y_flips = 0, 0
            distance, x_axis_distance, y_axis_distance, clicks, screenWidth, screenHeight, width, height, duration, inactivity, startedAt, clientStartedAt = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
            load_info = True
            scrolling = False
            
            for event in pageview:
                if load_info:
                    load_info = False
                    screenWidth, screenHeight, width, height, duration, inactivity, startedAt, clientStartedAt = event[0], event[1], event[2], event[3], event[4], event[5], event[6], event[7]
                        
                if event[2] == 'move':
                    if first_move_at == -1:
                        first_move_at = event[3][3]
                    last_move_at = event[3][3] + event[3][2]

                    xx = (event[3][1]['clientX']) / screenWidth * normalWidth
                    yy = (screenHeight - event[3][1]['clientY']) / screenHeight * normalHeight
                    coordinates = [xx, yy]
                    
                    x.append(xx)
                    y.append(yy)

                    if last_coordinates != [-1, -1]:
                        distance += distance_calculator(last_coordinates, coordinates)
                        x_axis_distance += abs(last_coordinates[0] - coordinates[0])
                        y_axis_distance += abs(last_coordinates[1] - coordinates[1])
                        x_increase, x_flips, y_increase, y_flips = flips_counter(coordinates, last_coordinates, x_increase, x_flips, y_increase, y_flips)

                    last_coordinates = coordinates

                if event[2] == 'click':
                    clicks += 1
                    bxx = (event[3][1]['clientX']) / screenWidth * normalWidth
                    byy = (screenHeight - event[3][1]['clientY']) / screenHeight * normalHeight
                    click_buttons.append([event[3][0], bxx, byy, len(x), event[3][3]])
                    
                if event[2] == 'scroll':
                    if event[3][0] > height:
                        scrolling = True
            
            time = last_move_at - first_move_at
            if time == 0:
                time = last_move_at
            if time == 0:
                time = -1

            velocity = distance / time
            x_axis_vel = x_axis_distance / time
            y_axis_vel = y_axis_distance / time
            
            initial_velocity = 0
            acc = (velocity - initial_velocity) / time
            x_axis_acc = (x_axis_vel - initial_velocity) / time
            y_axis_acc = (y_axis_vel - initial_velocity) / time
            
            last_ans_but_click = None
            last_next_but_click = None
            for but in click_buttons:
                if but[0] not in ["Ďalší", "Predchádzajúci"]:
                    last_ans_but_click = but
                if but[0] == "Ďalší":
                    last_next_but_click = but
                    
            if len(x) > 0 and last_next_but_click == None:
                last_next_but_click = ["Nezachytené kliknutie", x[-1], y[-1], len(x)]
            
            real_ideal_trajectory_diff = None
            max_deviation = None
            auc_ideal = 0
            react_time = 0
            
            if last_ans_but_click != None and last_next_but_click != None:
                real_ideal_trajectory_diff = distance_calculator([x[0], y[0]], [last_ans_but_click[1], last_ans_but_click[2]]) 
                real_ideal_trajectory_diff += distance_calculator([last_ans_but_click[1], last_ans_but_click[2]], [last_next_but_click[1], last_next_but_click[2]])
                real_ideal_trajectory_diff = distance - real_ideal_trajectory_diff
                auc_ideal = np.trapz(y=[y[0], y[last_ans_but_click[3] -1]], x=[x[0], x[last_ans_but_click[3] -1]]) 
                
                react_time = last_ans_but_click[4]
                
                p1 = np.asarray((x[0], y[0]))
                p2 = np.asarray((last_ans_but_click[1], last_ans_but_click[2]))
                max_deviation = 0
                # max_deviation from coordinates before final decision, use range(len(x)) to consider all coordinates
                for i in range(last_ans_but_click[3]):
                    p3 = np.asarray((x[i], y[i]))
                    d = norm(np.cross(p2-p1, p1-p3))/norm(p2-p1)
                    if d < 0:
                        raise "Error"
                    if d > max_deviation:
                        max_deviation = d
            
            # auc from all coordinates
            # auc_all = auc_calculator(y=y, x=x)
            auc_all = np.trapz(y=y, x=x)
            auc_diff = abs(auc_all - auc_ideal)
            
            if not silent:
                print(time, "ms")
                print(duration, 'ms - pageview duration')
                print(distance, "px")
                
                print(real_ideal_trajectory_diff, "px")
                print(max_deviation, "px")
                print(velocity, "px/ms")
                print(auc_diff, 'px^2')
                print(clicks, 'clicks')
                print(x_flips, 'x-flips')
                print(y_flips, 'y-flips')
                print(scrolling, 'scrolling')
                print("---------------------------------")
            time_arr.append(time)
            duration_arr.append(duration)
            distance_arr.append(distance)
            x_axis_distance_arr.append(x_axis_distance)
            y_axis_distance_arr.append(y_axis_distance)
            real_ideal_trajectory_diff_arr.append(real_ideal_trajectory_diff)
            max_deviation_arr.append(max_deviation)
            velocity_arr.append(velocity)
            x_axis_velocity_arr.append(x_axis_vel)
            y_axis_velocity_arr.append(y_axis_vel)
            acc_arr.append(acc)
            x_axis_acc_arr.append(x_axis_acc)
            y_axis_acc_arr.append(y_axis_acc)
            auc_arr.append(auc_diff)
            clicks_arr.append(clicks)
            x_flips_arr.append(x_flips)
            y_flips_arr.append(y_flips)
            scrolling_arr.append(scrolling)
            init_time_arr.append(first_move_at)
            react_time_arr.append(react_time)
    if len(time_arr) != 0:
        max_id = distance_arr.index(max(distance_arr))
        return {
                  page + '-time' : time_arr[max_id], 
                  page + '-duration' : duration_arr[max_id], 
                  page + '-distance' : distance_arr[max_id], 
                  page + '-x_axis_distance' : x_axis_distance_arr[max_id], 
                  page + '-y_axis_distance' : y_axis_distance_arr[max_id], 
                  page + '-real_ideal_trajectory_diff' : real_ideal_trajectory_diff_arr[max_id], 
                  page + '-max_deviation' : max_deviation_arr[max_id], 
                  page + '-velocity' : velocity_arr[max_id], 
                  page + '-x_axis_vel' : x_axis_velocity_arr[max_id], 
                  page + '-y_axis_vel' : y_axis_velocity_arr[max_id],
                  page + '-acc' : acc_arr[max_id], 
                  page + '-x_axis_acc' : x_axis_acc_arr[max_id], 
                  page + '-y_axis_acc' : y_axis_acc_arr[max_id],            
                  page + '-auc_diff' : auc_arr[max_id], 
                  page + '-clicks' : clicks_arr[max_id], 
                  page + '-x_flips' : x_flips_arr[max_id], 
                  page + '-y_flips' : y_flips_arr[max_id],
                  page + '-visits' : len(distance_arr),
                  page + '-scrolling': scrolling_arr[max_id],
                  page + '-init_time': init_time_arr[max_id],
                  page + '-react_time': react_time_arr[max_id]
                 }
    else:
        return {}

Definition of names of columns for mean and sum metrics for pageviews for Big5 questions and for Demographic questions. Creation of empty dataframe with columns for mean and sum metrics and metrics for individual pageviews.

In [8]:
metric_names = ['time', 'duration', 'distance', 'x_axis_distance', 'y_axis_distance', 'real_ideal_trajectory_diff', 'max_deviation', 'velocity', 
                'x_axis_vel', 'y_axis_vel', 'acc', 'x_axis_acc', 'y_axis_acc', 'auc_diff', 'clicks', 'x_flips', 'y_flips', 'visits', 'scrolling', 
                'init_time', 'react_time']    

column_names = ['Session'] + BIG5_MEAN_COLS + BIG5_SUM_COLS + DEM_MEAN_COLS + DEM_SUM_COLS

for x in range(68):
    for name in metric_names:
        column_names.append('Page_{}-{}'.format(x, name))

df = pd.DataFrame(columns = column_names)

Cycle for calling print_page_stats for every pageview in session and appending returned dictionary to dataframe. If sessions contains empty pageviews, session is not appended to the dataframe.

In [9]:
# for session_number in range(20):
for session_number in range(len(sessions_df)):

    session = sessions_df.iloc[[session_number]]
    print(session_number + 1, '/', len(sessions_df), '-', session['Session'][session_number])
    row = {'Session' : session['Session'][session_number]}
    
    append_flag = True
    
    for i in range(68):
        page = 'Page_{}'.format(i)
        # print(page)

        dic = print_page_stats(session, page, silent = True)
        if dic == {}:
            append_flag = False
            
        row = row | dic
        
    if append_flag:
        df = df.append(row, ignore_index=True)
# scatter_plot_from_coordinates(x, y, normalWidth, normalHeight)

1 / 165 - S5WLXErzUWNQqkDKnRC1j53PD


C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_18592\1892824478.py:98: RuntimeWarning: invalid value encountered in double_scalars
  d = norm(np.cross(p2-p1, p1-p3))/norm(p2-p1)


2 / 165 - gsd4FIhsnwaOIMOtoSqX4geVy
3 / 165 - q7Cvsx4iO2KeFdZj7kRY2Izr5
4 / 165 - IKW3ZZ9lcX5Z3vSKEghI73tuI
5 / 165 - UoBDwgoBWEavpKTmD0fwXax6z
6 / 165 - UEs1kb8w1KAzuLZMpFE3hzHGX
7 / 165 - ehP8b8PknsFj4ucqiKxdvuwU7
8 / 165 - ml3hCkdGvNgUCDKlhCqiZCeiD
9 / 165 - DrTk41pNckxJYIdilpta9qJBT


C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_18592\1892824478.py:98: RuntimeWarning: invalid value encountered in double_scalars
  d = norm(np.cross(p2-p1, p1-p3))/norm(p2-p1)


10 / 165 - jcvlt3renH3FwZuO0vUqmgbKt
11 / 165 - lq2tgTizx2nILy97FiBhLenuC
12 / 165 - 5bITkXEwKX7rJOTSfdcBrxtOo
13 / 165 - PQrxFN8wOy7kHNw0cGORMXIIq
14 / 165 - n8jJVTfZpJgGvx01wriH6LLKX
15 / 165 - k3qdYWdIZkboxvmar40fTzyDW
16 / 165 - AT8OFfwPJQZXnyNk1u3dUk2Vv
17 / 165 - xEoYPtO0Mbwjg5PkET57lc4HB
18 / 165 - QwE0GxlfqvIMNVeBPIWFdVSZE
19 / 165 - 93npiDWQK6VipNjEWLfWafqsr
20 / 165 - 0GNYlUU5mGtxOPKr275ut1hI7
21 / 165 - XOpFHITieEkY41l4GqKuDsv63
22 / 165 - MP9vzJRokFz5RJ02KSPxOWceB
23 / 165 - kDtZ55pukriLSh8VOnV6D5h3N
24 / 165 - bFWJvlSjZLbgznUva0d86yGwS
25 / 165 - EvzgBk8WwoQCCT00ySuBnvqEY
26 / 165 - J84AyKOw3eXAhQAE9iZYAYCxB
27 / 165 - vuUOqLBt9qjjelMZTMYHU7uer
28 / 165 - 34OsjvpQRQYY2xLKiMczMVFRE
29 / 165 - iHUU7ba0omNZO1f9xAGsOxLTZ
30 / 165 - VYV1nRhoEynhrmAnU8ouAzNS1
31 / 165 - KhBmctxEkh2gj805nC9od75PH
32 / 165 - XJnSu2wXLfz7WBqNLXrladCqv
33 / 165 - JJYXFCVzYwSzr21WonkAkHugU
34 / 165 - VajALjCapZgxxkAzTWNDVWXOB
35 / 165 - IL3mma1ZJyio3hujEo7ckDo60
36 / 165 - SYQzwFhlmLM1t92DwOj6sIuGC
3

...

88 / 165 - GIszPEY36M0Y6Zaq2mJQYE2Ev
89 / 165 - MHWlCKW1Rth7IufdKQ53gcHoW
90 / 165 - QJpzM76mOzzx41jlZ2cD68ZVr
91 / 165 - DPESrzxKVWTAlVhksRGpPRAKy
92 / 165 - sMjVKK5bH9bGGEeDF4TMra2xb
93 / 165 - fjAPfBwmai5If7wWXOgoMldv5


C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_18592\1892824478.py:98: RuntimeWarning: invalid value encountered in double_scalars
  d = norm(np.cross(p2-p1, p1-p3))/norm(p2-p1)


94 / 165 - Nhkvw5dbEwImQaB1qZNPqm7Qh
95 / 165 - 5HPhIkC2ryz2x5IiTSDGramvs
96 / 165 - v12R8blPbUmMEeidxzOBHB8Sg
97 / 165 - RBPuYafOHOIdMXLQ07pBerkXl
98 / 165 - Nep6DLKgmRWkY7UkSVonr9BWk
99 / 165 - Mty79oNR1egcGcZepG1PeSQzz
100 / 165 - B9f22juxHTJ70W0mINyPifnCJ
101 / 165 - Wngebc5UMfE72xGINkEuiBh9W
102 / 165 - 0Owc70OVu6pVzCQiUSfQ9qZPN
103 / 165 - hIii8Pf2YJD2ckZEkRd4V1Fhl
104 / 165 - L40JDl98uqsf8CenfVAVi1Lvn
105 / 165 - QiC4U4uGfiAo6e0wy24FdFIhd
106 / 165 - LjLVXgA3apikdQjRjHTjSwWar
107 / 165 - UAytqC2udrP0JcwjdcjhhCNw8
108 / 165 - 9S3HZRuFZMdMNfCcgDyd6VdKW
109 / 165 - lsJDzWO4z8U4vu9lEG9D7iuEp
110 / 165 - Ot4TGHaqVnliCfLgnDqsL6veT
111 / 165 - X3UTDLu8C6PzOEvMgh07iezGe
112 / 165 - u5eFRqSdth3QS5GCom6kO06Hm
113 / 165 - pfRzmwXqNCWKOOjeCLdUafF79
114 / 165 - xGZiyHsLw13YSV8NwVLnevNK8
115 / 165 - D1gKbSiqnXpQf529U37JIxW3n
116 / 165 - hHka5rTzO7RgOjivQH0JjlOhO
117 / 165 - zjMG8tuXTouFw6tDCP7oOVHSh
118 / 165 - rE94Xg0xQb1ELPd2kKJBKqPK8
119 / 165 - pjbcxDZTIqBgwJId1zjBVZQbS
120 / 165 - 2wQp5R

C:\Users\PeterSmrecek\AppData\Local\Temp\ipykernel_18592\1892824478.py:98: RuntimeWarning: invalid value encountered in double_scalars
  d = norm(np.cross(p2-p1, p1-p3))/norm(p2-p1)


146 / 165 - OUCJrEdoxEfok2ehKMHEIUPYy
147 / 165 - BiOFEa9J0n65xUQoudAq2hJ0G
148 / 165 - 6puGRNWRQm2UeBn6DOoAE7sT6
149 / 165 - aotIxinfc0vHzEi9ehYB7OhuI
150 / 165 - uwMvzBSDsgBHsj50M3MLQmOaw
151 / 165 - kWXMrfT5jl0TFxhgstlFq9PkE
152 / 165 - Y13Fl371AORoLNMryjjht5lCo
153 / 165 - tseRJkAbZhFUXHnhjPWHBhHmF
154 / 165 - uLYaRHExVIP8GhfUrmzuWP5Pb
155 / 165 - OK0LALqyDvRa2oBp6nnkjdks9
156 / 165 - 99TUULTfZzD3H9F0bzzHus3c4
157 / 165 - KDWak467FmFZN321qc8Qumwhc
158 / 165 - wGP5zHu6DVsA0urhzP9fESCWP
159 / 165 - eCbJ8XLbi6usVJHzuJvVHREIN
160 / 165 - XHhPFBsnlOXgdBESumLQeLMQs
161 / 165 - bHA4rnvBgjkkSZYmEUAuwUHO8
162 / 165 - kIUPhdYJKjciyg8Mm1SptAVKC
163 / 165 - aACGAcEw7Bw5k2tpl32WRnazk
164 / 165 - xJmCJsSVnx5RXXi90tWMr1t9x
165 / 165 - yQOKd4r41wCPrKd5cVFaAuJED


Showing generated dataframe

In [10]:
len(df)

138

In [11]:
df[:10]

Session big5_mean_time big5_mean_duration  \
0  S5WLXErzUWNQqkDKnRC1j53PD            NaN                NaN   
1  gsd4FIhsnwaOIMOtoSqX4geVy            NaN                NaN   
2  IKW3ZZ9lcX5Z3vSKEghI73tuI            NaN                NaN   
3  DrTk41pNckxJYIdilpta9qJBT            NaN                NaN   
4  jcvlt3renH3FwZuO0vUqmgbKt            NaN                NaN   
5  PQrxFN8wOy7kHNw0cGORMXIIq            NaN                NaN   
6  k3qdYWdIZkboxvmar40fTzyDW            NaN                NaN   
7  xEoYPtO0Mbwjg5PkET57lc4HB            NaN                NaN   
8  93npiDWQK6VipNjEWLfWafqsr            NaN                NaN   
9  0GNYlUU5mGtxOPKr275ut1hI7            NaN                NaN   

  big5_mean_distance big5_mean_x_axis_distance big5_mean_y_axis_distance  \
0                NaN                       NaN                       NaN   
1                NaN                       NaN                       NaN   
2                NaN                       NaN                       NaN   
3                NaN                       NaN                       NaN   
4                NaN                       NaN                       NaN   
5                NaN                       NaN                       NaN   
6                NaN                       NaN                       NaN   
7                NaN                       NaN                       NaN   
8                NaN                       NaN                       NaN   
9                NaN                       NaN                       NaN   

  big5_mean_real_ideal_trajectory_diff big5_mean_max_deviation  \
0                                  NaN                     NaN   
1                                  NaN                     NaN   
2                                  NaN                     NaN   
3                                  NaN                     NaN   
4                                  NaN                     NaN   
5                                  NaN                     NaN   
6                                  NaN                     NaN   
7                                  NaN                     NaN   
8                                  NaN                     NaN   
9                                  NaN                     NaN   

  big5_mean_velocity big5_mean_x_axis_vel big5_mean_y_axis_vel big5_mean_acc  \
0                NaN                  NaN                  NaN           NaN   
1                NaN                  NaN                  NaN           NaN   
2                NaN                  NaN                  NaN           NaN   
3                NaN                  NaN                  NaN           NaN   
4                NaN                  NaN                  NaN           NaN   
5                NaN                  NaN                  NaN           NaN   
6                NaN                  NaN                  NaN           NaN   
7                NaN                  NaN                  NaN           NaN   
8                NaN                  NaN                  NaN           NaN   
9                NaN                  NaN                  NaN           NaN   

  big5_mean_x_axis_acc big5_mean_y_axis_acc big5_mean_auc_diff  \
0                  NaN                  NaN                NaN   
1                  NaN                  NaN                NaN   
2                  NaN                  NaN                NaN   
3                  NaN                  NaN                NaN   
4                  NaN                  NaN                NaN   
5                  NaN                  NaN                NaN   
6                  NaN                  NaN                NaN   
7                  NaN                  NaN                NaN   
8                  NaN                  NaN                NaN   
9                  NaN                  NaN                NaN   

  big5_mean_clicks big5_mean_x_flips big5_mean_y_flips big5_mean_init_time  \
0              NaN               NaN        

Function for generating names of columns corresponding with given range of pages and given names of metrics.

In [12]:
def generate_names_of_columns(rng_pages, metrics):
    selected_columns = []
    
    for name in metrics:
        one_metric = []
        for x in rng_pages:
            one_metric.append('Page_{}-{}'.format(x, name))
        selected_columns.append(one_metric)
    return selected_columns

Function for calculating secondary (mean and sum) metrics from individual pageviews metrics. Function generates names of corresponding columns and perform operation on those columns, saving result into dataframe.

In [13]:
def calculate_secondary_metrics(df, rng_pages, df_mean_cols, df_sum_cols, metric_names_mean, metric_names_sum):
    generated_names_mean = generate_names_of_columns(rng_pages, metric_names_mean)
    generated_names_sum = generate_names_of_columns(rng_pages, metric_names_sum)
    
    for x in range(len(df_mean_cols)):
        print('Pairing', df_mean_cols[x], 'with', generated_names_mean[x])
        df[df_mean_cols[x]] = df[generated_names_mean[x]].mean(axis=1)
    print()    
    for x in range(len(df_sum_cols)):
        print('Pairing', df_sum_cols[x], 'with', generated_names_sum[x])
        df[df_sum_cols[x]] = df[generated_names_sum[x]].sum(axis=1)

Definition of mean metrics, sum metrics and ranges of pages corresponding with Big5 questions and Demographics questions

In [14]:
metric_names_mean = ['time', 'duration', 'distance', 'x_axis_distance','y_axis_distance', 'real_ideal_trajectory_diff', 'max_deviation', 
                    'velocity', 'x_axis_vel', 'y_axis_vel', 'acc', 'x_axis_acc', 'y_axis_acc', 'auc_diff', 'clicks', 'x_flips', 'y_flips',
                    'init_time', 'react_time']   

metric_names_sum = ['visits', 'scrolling', 'x_flips', 'y_flips']   

big5_pages = range(3, 51)
dem_pages = range(52, 66)

Calculate secondary metrics for Big5 questions.

In [15]:
calculate_secondary_metrics(df, big5_pages, BIG5_MEAN_COLS, BIG5_SUM_COLS, metric_names_mean, metric_names_sum)

Pairing big5_mean_time with ['Page_3-time', 'Page_4-time', 'Page_5-time', 'Page_6-time', 'Page_7-time', 'Page_8-time', 'Page_9-time', 'Page_10-time', 'Page_11-time', 'Page_12-time', 'Page_13-time', 'Page_14-time', 'Page_15-time', 'Page_16-time', 'Page_17-time', 'Page_18-time', 'Page_19-time', 'Page_20-time', 'Page_21-time', 'Page_22-time', 'Page_23-time', 'Page_24-time', 'Page_25-time', 'Page_26-time', 'Page_27-time', 'Page_28-time', 'Page_29-time', 'Page_30-time', 'Page_31-time', 'Page_32-time', 'Page_33-time', 'Page_34-time', 'Page_35-time', 'Page_36-time', 'Page_37-time', 'Page_38-time', 'Page_39-time', 'Page_40-time', 'Page_41-time', 'Page_42-time', 'Page_43-time', 'Page_44-time', 'Page_45-time', 'Page_46-time', 'Page_47-time', 'Page_48-time', 'Page_49-time', 'Page_50-time']
Pairing big5_mean_duration with ['Page_3-duration', 'Page_4-duration', 'Page_5-duration', 'Page_6-duration', 'Page_7-duration', 'Page_8-duration', 'Page_9-duration', 'Page_10-duration', 'Page_11-duration', 'Pag

Calculate secondary metrics for Demographics questions.

In [16]:
calculate_secondary_metrics(df, dem_pages, DEM_MEAN_COLS, DEM_SUM_COLS, metric_names_mean, metric_names_sum)

Pairing dem_mean_time with ['Page_52-time', 'Page_53-time', 'Page_54-time', 'Page_55-time', 'Page_56-time', 'Page_57-time', 'Page_58-time', 'Page_59-time', 'Page_60-time', 'Page_61-time', 'Page_62-time', 'Page_63-time', 'Page_64-time', 'Page_65-time']
Pairing dem_mean_duration with ['Page_52-duration', 'Page_53-duration', 'Page_54-duration', 'Page_55-duration', 'Page_56-duration', 'Page_57-duration', 'Page_58-duration', 'Page_59-duration', 'Page_60-duration', 'Page_61-duration', 'Page_62-duration', 'Page_63-duration', 'Page_64-duration', 'Page_65-duration']
Pairing dem_mean_distance with ['Page_52-distance', 'Page_53-distance', 'Page_54-distance', 'Page_55-distance', 'Page_56-distance', 'Page_57-distance', 'Page_58-distance', 'Page_59-distance', 'Page_60-distance', 'Page_61-distance', 'Page_62-distance', 'Page_63-distance', 'Page_64-distance', 'Page_65-distance']
Pairing dem_mean_x_axis_distance with ['Page_52-x_axis_distance', 'Page_53-x_axis_distance', 'Page_54-x_axis_distance', 'Pag

Ordering columns and showing genrated data in dataframe.

In [17]:
ordered_columns = ['Session']

for x in range(len(BIG5_MEAN_COLS)):
    ordered_columns.append(BIG5_MEAN_COLS[x])
    ordered_columns.append(DEM_MEAN_COLS[x])

for x in range(len(BIG5_SUM_COLS)):
    ordered_columns.append(BIG5_SUM_COLS[x])
    ordered_columns.append(DEM_SUM_COLS[x])

In [18]:
df[ordered_columns][:10]

,Session,big5_mean_time,dem_mean_time,big5_mean_duration,dem_mean_duration,big5_mean_distance,dem_mean_distance,big5_mean_x_axis_distance,dem_mean_x_axis_distance,big5_mean_y_axis_distance,dem_mean_y_axis_distance,big5_mean_real_ideal_trajectory_diff,dem_mean_real_ideal_trajectory_diff,big5_mean_max_deviation,dem_mean_max_deviation,big5_mean_velocity,dem_mean_velocity,big5_mean_x_axis_vel,dem_mean_x_axis_vel,big5_mean_y_axis_vel,dem_mean_y_axis_vel,big5_mean_acc,dem_mean_acc,big5_mean_x_axis_acc,dem_mean_x_axis_acc,big5_mean_y_axis_acc,dem_mean_y_axis_acc,big5_mean_auc_diff,dem_mean_auc_diff,big5_mean_clicks,dem_mean_clicks,big5_mean_x_flips,dem_mean_x_flips,big5_mean_y_flips,dem_mean_y_flips,big5_mean_init_time,dem_mean_init_time,big5_mean_react_time,dem_mean_react_time,big5_sum_visits,dem_sum_visits,big5_sum_scrolling,dem_sum_scrolling,big5_sum_x_flips,dem_sum_x_flips,big5_sum_y_flips,dem_sum_y_flips
0,S5WLXErzUWNQqkDKnRC1j53PD,927.687500,968.142857,1129.958333,1172.928571,687.662935,676.131010,539.821777,537.631138,352.864583,349.553571,315.013898,259.478873,11.210961,8.109346,0.751742,0.685566,0.583339,0.536824,0.395297,0.365599,0.000859,0.000721,0.000656,0.000557,0.000464,0.000396,106455.128479,93085.159738,2.062500,2.000000,1.458333,1.285714,1.604167,1.571429,192.770833,197.928571,332.395833,360.642857,49.0,14.0,0.0,0.0,70.0,18.0,77.0,22.0
1,gsd4FIhsnwaOIMOtoSqX4geVy,2861.604167,2596.928571,4713.104167,4311.571429,855.009425,774.897241,685.480469,632.524554,372.833333,335.571429,304.349121,321.803818,66.593651,95.561074,0.373892,0.331024,0.298735,0.259609,0.170425,0.159105,0.000231,0.000182,0.000180,0.000137,0.000112,0.000098,110704.204102,93036.612723,2.250000,2.214286,2.354167,2.285714,2.645833,2.285714,1531.083333,1284.500000,3692.000000,3136.142857,48.0,14.0,0.0,0.0,113.0,32.0,127.0,32.0
2,IKW3ZZ9lcX5Z3vSKEghI73tuI,4072.166667,3355.428571,6049.333333,4910.928571,1271.899270,1114.092414,1062.445312,923.705357,498.375000,427.971429,622.502322,396.388659,179.196994,109.116820,0.397325,0.373652,0.339513,0.308606,0.150907,0.148381,0.000165,0.000153,0.000143,0.000125,0.000061,0.000063,97427.068750,117154.419643,2.375000,2.214286,3.791667,3.714286,3.208333,2.714286,1664.583333,1198.357143,4890.312500,3495.785714,49.0,14.0,0.0,0.0,182.0,52.0,154.0,38.0
3,DrTk41pNckxJYIdilpta9qJBT,4632.333333,3403.214286,5442.458333,4672.142857,1689.802317,1445.463065,1425.574219,1249.064732,600.791667,496.571429,1043.191131,895.447005,234.883818,205.830120,0.413681,0.418111,0.344645,0.357633,0.154020,0.149203,0.000124,0.000146,0.000102,0.000124,0.000049,0.000054,122730.759766,100795.295759,2.291667,2.142857,5.229167,3.571429,5.895833,3.428571,524.354167,895.928571,4230.395833,3408.714286,49.0,15.0,0.0,0.0,251.0,50.0,283.0,48.0
4,jcvlt3renH3FwZuO0vUqmgbKt,2819.625000,3208.642857,5388.916667,5244.500000,933.318823,1045.206720,731.521484,876.488839,439.187500,421.285714,414.481089,496.021194,81.647968,81.164914,0.448751,0.451040,0.342532,0.368974,0.226396,0.197662,0.000291,0.000269,0.000214,0.000220,0.000156,0.000120,95816.069336,112468.051339,2.083333,2.071429,3.729167,3.071429,2.895833,2.857143,2220.812500,1449.642857,4348.145833,3976.785714,52.0,16.0,0.0,0.0,179.0,43.0,139.0,40.0
5,PQrxFN8wOy7kHNw0cGORMXIIq,3466.729167,3508.714286,4973.979167,5248.285714,613.717068,751.662245,438.474609,567.482143,325.645833,391.142857,165.354949,174.867433,69.496165,45.105476,0.204544,0.290157,0.141582,0.218095,0.114487,0.153589,0.000086,0.000141,0.000057,0.000106,0.000052,0.000076,77241.366211,116631.907366,2.125000,2.142857,2.416667,2.500000,2.145833,3.071429,1192.312500,1347.428571,3859.062500,4049.285714,48.0,14.0,0.0,0.0,116.0,35.0,103.0,43.0
6,k3qdYWdIZkboxvmar40fTzyDW,3652.562500,4709.071429,4708.895833,5968.857143,716.152106,672.877841,555.908203,514.078125,341.145833,332.857143,262.572759,175.779554,114.950822,55.785642,0.229458,0.197965,0.177295,0.149977,0.111842,0.098789,0.000091,0.000070,0.000070,0.000053,0.000045,0.000036,78786.382812,98956.024

Function for showing data in dataframe. Showing only selected metrics and only from selected pages for easy debugging.

In [19]:
def print_one_metric(df, metrics, first_page, last_page, show):
    metric_names = ['Session']
    for x in range(first_page, last_page):
        for metric in metrics:
            metric_names.append('Page_{}-{}'.format(x, metric))

    return df[metric_names][:show]

In [20]:
first_page, last_page, show = 3, 66, 10

Showing simmilar metrics together.

In [21]:
print_one_metric(df, ['time', 'duration'], first_page, last_page, show)

,Session,Page_3-time,Page_3-duration,Page_4-time,Page_4-duration,Page_5-time,Page_5-duration,Page_6-time,Page_6-duration,Page_7-time,Page_7-duration,Page_8-time,Page_8-duration,Page_9-time,Page_9-duration,Page_10-time,Page_10-duration,Page_11-time,Page_11-duration,Page_12-time,Page_12-duration,Page_13-time,Page_13-duration,Page_14-time,Page_14-duration,Page_15-time,Page_15-duration,Page_16-time,Page_16-duration,Page_17-time,Page_17-duration,Page_18-time,Page_18-duration,Page_19-time,Page_19-duration,Page_20-time,Page_20-duration,Page_21-time,Page_21-duration,Page_22-time,Page_22-duration,Page_23-time,Page_23-duration,Page_24-time,Page_24-duration,Page_25-time,Page_25-duration,Page_26-time,Page_26-duration,Page_27-time,Page_27-duration,Page_28-time,Page_28-duration,Page_29-time,Page_29-duration,Page_30-time,Page_30-duration,Page_31-time,Page_31-duration,Page_32-time,Page_32-duration,Page_33-time,Page_33-duration,Page_34-time,Page_34-duration,Page_35-time,Page_35-duration,Page_36-time,Page_36-duration,Page_37-time,Page_37-duration,Page_38-time,Page_38-duration,Page_39-time,Page_39-duration,Page_40-time,Page_40-duration,Page_41-time,Page_41-duration,Page_42-time,Page_42-duration,Page_43-time,Page_43-duration,Page_44-time,Page_44-duration,Page_45-time,Page_45-duration,Page_46-time,Page_46-duration,Page_47-time,Page_47-duration,Page_48-time,Page_48-duration,Page_49-time,Page_49-duration,Page_50-time,Page_50-duration,Page_51-time,Page_51-duration,Page_52-time,Page_52-duration,Page_53-time,Page_53-duration,Page_54-time,Page_54-duration,Page_55-time,Page_55-duration,Page_56-time,Page_56-duration,Page_57-time,Page_57-duration,Page_58-time,Page_58-duration,Page_59-time,Page_59-duration,Page_60-time,Page_60-duration,Page_61-time,Page_61-duration,Page_62-time,Page_62-duration,Page_63-time,Page_63-duration,Page_64-time,Page_64-duration,Page_65-time,Page_65-duration
0,S5WLXErzUWNQqkDKnRC1j53PD,816,1607,819,1066,637,876,811,948,799,1091,918,1037,1022,1230,2475,2622,798,1087,1183,1350,1093,1224,838,968,649,957,770,1029,994,1155,985,1105,745,1010,717,972,886,1020,963,1087,1149,1290,1457,1607,662,981,699,1005,1150,1277,983,1141,850,1008,896,1029,982,1120,1043,1189,1054,1175,834,978,883,1030,738,1021,997,1118,985,1305,917,1083,751,1037,927,1049,910,1049,1045,1198,896,1048,717,1003,808,966,914,1045,979,1118,727,989,658,938,641,1097,1383,1794,734,1068,832,972,999,1139,1080,1207,1064,1220,845,993,977,1119,806,1090,1046,1194,1133,1284,684,1085,986,1117,985,1139
1,gsd4FIhsnwaOIMOtoSqX4geVy,14236,14893,2448,4529,4051,4355,1432,6039,3648,4513,4566,5363,821,3128,3224,3628,4792,5444,5674,5820,2568,2723,1657,3820,1717,3344,1554,3825,918,4785,4399,4738,1965,3155,1358,2893,1001,4387,918,4348,816,3439,3967,5036,2249,3359,1287,5793,1750,4351,3261,3952,1758,3476,2094,3625,1099,5745,5388,5963,1031,3455,2136,2405,2551,4114,4465,5015,1586,4054,1958,3626,2194,4382,1342,3145,1371,5546,8230,10889,1337,6925,1509,3294,1553,5313,5286,6503,5266,6267,3533,4211,2728,3243,2665,3373,404,8126,1559,3392,2381,3775,2156,3009,982,2558,1699,3003,2387,6584,3349,3903,2605,4342,1539,5068,3983,4838,8771,8922,1028,2622,2666,3306,1252,5040
2,IKW3ZZ9lcX5Z3vSKEghI73tuI,8045,10391,7037,8913,10696,11859,5782,6373,5984,6148,2184,4112,1794,4089,21007,23125,6415,7362,3594,6026,5194,6741,1948,3141,1876,3101,3557,4914,3038,4473,1260,4022,2416,3957,1301,3781,1664,3839,8996,14308,1510,3741,3064,4257,3965,5015,3859,4348,1611,3544,2948,5262,2505,14647,4984,7305,2133,7017,2271,3371,5105,5695,1975,4047,2456,3381,4310,7061,1891,4016,2632,3823,3743,4634,4032,5877,1502,3416,6283,6755,1907,5572,2171,3064,3856,4608,3500,6356,3758,4624,3781,5860,2898,4069,7026,8328,7325,7735,6716,8075,4622,5559,1588,2418,2057,3255,3239,4269,4768,6432,1395,2994,2932,5620,2175,5392,3276,3895,4153,7115,1919,3368,2289,3016,5847,7345
3,DrTk41pNckxJYIdilpta9qJBT,8602,9490,3946,4866,3760,4615,4609,5031,5683,5840,3568,3714,3060,3420,4673,5033,6321,6594,3703,4312,2619,3351,5302,5913,3437,4276,3487,4921,2601,3837,1400,3246,1399,3336

In [22]:
print_one_metric(df, ['distance', 'x_axis_distance','y_axis_distance'], first_page, last_page, show)

,Session,Page_3-distance,Page_3-x_axis_distance,Page_3-y_axis_distance,Page_4-distance,Page_4-x_axis_distance,Page_4-y_axis_distance,Page_5-distance,Page_5-x_axis_distance,Page_5-y_axis_distance,Page_6-distance,Page_6-x_axis_distance,Page_6-y_axis_distance,Page_7-distance,Page_7-x_axis_distance,Page_7-y_axis_distance,Page_8-distance,Page_8-x_axis_distance,Page_8-y_axis_distance,Page_9-distance,Page_9-x_axis_distance,Page_9-y_axis_distance,Page_10-distance,Page_10-x_axis_distance,Page_10-y_axis_distance,Page_11-distance,Page_11-x_axis_distance,Page_11-y_axis_distance,Page_12-distance,Page_12-x_axis_distance,Page_12-y_axis_distance,Page_13-distance,Page_13-x_axis_distance,Page_13-y_axis_distance,Page_14-distance,Page_14-x_axis_distance,Page_14-y_axis_distance,Page_15-distance,Page_15-x_axis_distance,Page_15-y_axis_distance,Page_16-distance,Page_16-x_axis_distance,Page_16-y_axis_distance,Page_17-distance,Page_17-x_axis_distance,Page_17-y_axis_distance,Page_18-distance,Page_18-x_axis_distance,Page_18-y_axis_distance,Page_19-distance,Page_19-x_axis_distance,Page_19-y_axis_distance,Page_20-distance,Page_20-x_axis_distance,Page_20-y_axis_distance,Page_21-distance,Page_21-x_axis_distance,Page_21-y_axis_distance,Page_22-distance,Page_22-x_axis_distance,Page_22-y_axis_distance,Page_23-distance,Page_23-x_axis_distance,Page_23-y_axis_distance,Page_24-distance,Page_24-x_axis_distance,Page_24-y_axis_distance,Page_25-distance,Page_25-x_axis_distance,Page_25-y_axis_distance,Page_26-distance,Page_26-x_axis_distance,Page_26-y_axis_distance,Page_27-distance,Page_27-x_axis_distance,Page_27-y_axis_distance,Page_28-distance,Page_28-x_axis_distance,Page_28-y_axis_distance,Page_29-distance,Page_29-x_axis_distance,Page_29-y_axis_distance,Page_30-distance,Page_30-x_axis_distance,Page_30-y_axis_distance,Page_31-distance,Page_31-x_axis_distance,Page_31-y_axis_distance,Page_32-distance,Page_32-x_axis_distance,Page_32-y_axis_distance,Page_33-distance,Page_33-x_axis_distance,Page_33-y_axis_distance,Page_34-distance,Page_34-x_axis_distance,Page_34-y_axis_distance,Page_35-distance,Page_35-x_axis_distance,Page_35-y_axis_distance,Page_36-distance,Page_36-x_axis_distance,Page_36-y_axis_distance,Page_37-distance,Page_37-x_axis_distance,Page_37-y_axis_distance,Page_38-distance,Page_38-x_axis_distance,Page_38-y_axis_distance,Page_39-distance,Page_39-x_axis_distance,Page_39-y_axis_distance,Page_40-distance,Page_40-x_axis_distance,Page_40-y_axis_distance,Page_41-distance,Page_41-x_axis_distance,Page_41-y_axis_distance,Page_42-distance,Page_42-x_axis_distance,Page_42-y_axis_distance,Page_43-distance,Page_43-x_axis_distance,Page_43-y_axis_distance,Page_44-distance,Page_44-x_axis_distance,Page_44-y_axis_distance,Page_45-distance,Page_45-x_axis_distance,Page_45-y_axis_distance,Page_46-distance,Page_46-x_axis_distance,Page_46-y_axis_distance,Page_47-distance,Page_47-x_axis_distance,Page_47-y_axis_distance,Page_48-distance,Page_48-x_axis_distance,Page_48-y_axis_distance,Page_49-distance,Page_49-x_axis_distance,Page_49-y_axis_distance,Page_50-distance,Page_50-x_axis_distance,Page_50-y_axis_distance,Page_51-distance,Page_51-x_axis_distance,Page_51-y_axis_distance,Page_52-distance,Page_52-x_axis_distance,Page_52-y_axis_distance,Page_53-distance,Page_53-x_axis_distance,Page_53-y_axis_distance,Page_54-distance,Page_54-x_axis_distance,Page_54-y_axis_distance,Page_55-distance,Page_55-x_axis_distance,Page_55-y_axis_distance,Page_56-distance,Page_56-x_axis_distance,Page_56-y_axis_distance,Page_57-distance,Page_57-x_axis_distance,Page_57-y_axis_distance,Page_58-distance,Page_58-x_axis_distance,Page_58-y_axis_distance,Page_59-distance,Page_59-x_axis_distance,Page_59-y_axis_distance,Page_60-distance,Page_60-x_axis_distance,Page_60-y_axis_distance,Page_61-distance,Page_61-x_axis_distance,Page_61-y_axis_distance,Page_62-distance,Page_62-x_axis_distance,Page_62-y_axis_distance,Page_63-distance,Page_63-x_axis_distance,Page_63-y_axis_distance,Page_64-distance,Page_64-x_axis_distance,Page_6

In [23]:
print_one_metric(df, ['real_ideal_trajectory_diff', 'max_deviation'], first_page, last_page, show)

,Session,Page_3-real_ideal_trajectory_diff,Page_3-max_deviation,Page_4-real_ideal_trajectory_diff,Page_4-max_deviation,Page_5-real_ideal_trajectory_diff,Page_5-max_deviation,Page_6-real_ideal_trajectory_diff,Page_6-max_deviation,Page_7-real_ideal_trajectory_diff,Page_7-max_deviation,Page_8-real_ideal_trajectory_diff,Page_8-max_deviation,Page_9-real_ideal_trajectory_diff,Page_9-max_deviation,Page_10-real_ideal_trajectory_diff,Page_10-max_deviation,Page_11-real_ideal_trajectory_diff,Page_11-max_deviation,Page_12-real_ideal_trajectory_diff,Page_12-max_deviation,Page_13-real_ideal_trajectory_diff,Page_13-max_deviation,Page_14-real_ideal_trajectory_diff,Page_14-max_deviation,Page_15-real_ideal_trajectory_diff,Page_15-max_deviation,Page_16-real_ideal_trajectory_diff,Page_16-max_deviation,Page_17-real_ideal_trajectory_diff,Page_17-max_deviation,Page_18-real_ideal_trajectory_diff,Page_18-max_deviation,Page_19-real_ideal_trajectory_diff,Page_19-max_deviation,Page_20-real_ideal_trajectory_diff,Page_20-max_deviation,Page_21-real_ideal_trajectory_diff,Page_21-max_deviation,Page_22-real_ideal_trajectory_diff,Page_22-max_deviation,Page_23-real_ideal_trajectory_diff,Page_23-max_deviation,Page_24-real_ideal_trajectory_diff,Page_24-max_deviation,Page_25-real_ideal_trajectory_diff,Page_25-max_deviation,Page_26-real_ideal_trajectory_diff,Page_26-max_deviation,Page_27-real_ideal_trajectory_diff,Page_27-max_deviation,Page_28-real_ideal_trajectory_diff,Page_28-max_deviation,Page_29-real_ideal_trajectory_diff,Page_29-max_deviation,Page_30-real_ideal_trajectory_diff,Page_30-max_deviation,Page_31-real_ideal_trajectory_diff,Page_31-max_deviation,Page_32-real_ideal_trajectory_diff,Page_32-max_deviation,Page_33-real_ideal_trajectory_diff,Page_33-max_deviation,Page_34-real_ideal_trajectory_diff,Page_34-max_deviation,Page_35-real_ideal_trajectory_diff,Page_35-max_deviation,Page_36-real_ideal_trajectory_diff,Page_36-max_deviation,Page_37-real_ideal_trajectory_diff,Page_37-max_deviation,Page_38-real_ideal_trajectory_diff,Page_38-max_deviation,Page_39-real_ideal_trajectory_diff,Page_39-max_deviation,Page_40-real_ideal_trajectory_diff,Page_40-max_deviation,Page_41-real_ideal_trajectory_diff,Page_41-max_deviation,Page_42-real_ideal_trajectory_diff,Page_42-max_deviation,Page_43-real_ideal_trajectory_diff,Page_43-max_deviation,Page_44-real_ideal_trajectory_diff,Page_44-max_deviation,Page_45-real_ideal_trajectory_diff,Page_45-max_deviation,Page_46-real_ideal_trajectory_diff,Page_46-max_deviation,Page_47-real_ideal_trajectory_diff,Page_47-max_deviation,Page_48-real_ideal_trajectory_diff,Page_48-max_deviation,Page_49-real_ideal_trajectory_diff,Page_49-max_deviation,Page_50-real_ideal_trajectory_diff,Page_50-max_deviation,Page_51-real_ideal_trajectory_diff,Page_51-max_deviation,Page_52-real_ideal_trajectory_diff,Page_52-max_deviation,Page_53-real_ideal_trajectory_diff,Page_53-max_deviation,Page_54-real_ideal_trajectory_diff,Page_54-max_deviation,Page_55-real_ideal_trajectory_diff,Page_55-max_deviation,Page_56-real_ideal_trajectory_diff,Page_56-max_deviation,Page_57-real_ideal_trajectory_diff,Page_57-max_deviation,Page_58-real_ideal_trajectory_diff,Page_58-max_deviation,Page_59-real_ideal_trajectory_diff,Page_59-max_deviation,Page_60-real_ideal_trajectory_diff,Page_60-max_deviation,Page_61-real_ideal_trajectory_diff,Page_61-max_deviation,Page_62-real_ideal_trajectory_diff,Page_62-max_deviation,Page_63-real_ideal_trajectory_diff,Page_63-max_deviation,Page_64-real_ideal_trajectory_diff,Page_64-max_deviation,Page_65-real_ideal_trajectory_diff,Page_65-max_deviation
0,S5WLXErzUWNQqkDKnRC1j53PD,201.157349,0,162.537787,0,231.561288,0,565.961453,0,473.933526,0,243.197912,0,196.464719,0,1213.576216,355.863663,512.544937,0,10.615316,0,298.498805,4.698337,199.669431,2.5,230.816882,0,451.926073,0,457.042004,14.227104,341.52884,8.844115,159.322119,0,246.349149,0,473.060209,0,368.763011,0.901719,447.65953,1.165403,216.676798,6.548979,246.78822,0,423.312152,0,455.874743,1.25,273.802

In [24]:
print_one_metric(df, ['velocity', 'x_axis_vel', 'y_axis_vel'], first_page, last_page, show)

,Session,Page_3-velocity,Page_3-x_axis_vel,Page_3-y_axis_vel,Page_4-velocity,Page_4-x_axis_vel,Page_4-y_axis_vel,Page_5-velocity,Page_5-x_axis_vel,Page_5-y_axis_vel,Page_6-velocity,Page_6-x_axis_vel,Page_6-y_axis_vel,Page_7-velocity,Page_7-x_axis_vel,Page_7-y_axis_vel,Page_8-velocity,Page_8-x_axis_vel,Page_8-y_axis_vel,Page_9-velocity,Page_9-x_axis_vel,Page_9-y_axis_vel,Page_10-velocity,Page_10-x_axis_vel,Page_10-y_axis_vel,Page_11-velocity,Page_11-x_axis_vel,Page_11-y_axis_vel,Page_12-velocity,Page_12-x_axis_vel,Page_12-y_axis_vel,Page_13-velocity,Page_13-x_axis_vel,Page_13-y_axis_vel,Page_14-velocity,Page_14-x_axis_vel,Page_14-y_axis_vel,Page_15-velocity,Page_15-x_axis_vel,Page_15-y_axis_vel,Page_16-velocity,Page_16-x_axis_vel,Page_16-y_axis_vel,Page_17-velocity,Page_17-x_axis_vel,Page_17-y_axis_vel,Page_18-velocity,Page_18-x_axis_vel,Page_18-y_axis_vel,Page_19-velocity,Page_19-x_axis_vel,Page_19-y_axis_vel,Page_20-velocity,Page_20-x_axis_vel,Page_20-y_axis_vel,Page_21-velocity,Page_21-x_axis_vel,Page_21-y_axis_vel,Page_22-velocity,Page_22-x_axis_vel,Page_22-y_axis_vel,Page_23-velocity,Page_23-x_axis_vel,Page_23-y_axis_vel,Page_24-velocity,Page_24-x_axis_vel,Page_24-y_axis_vel,Page_25-velocity,Page_25-x_axis_vel,Page_25-y_axis_vel,Page_26-velocity,Page_26-x_axis_vel,Page_26-y_axis_vel,Page_27-velocity,Page_27-x_axis_vel,Page_27-y_axis_vel,Page_28-velocity,Page_28-x_axis_vel,Page_28-y_axis_vel,Page_29-velocity,Page_29-x_axis_vel,Page_29-y_axis_vel,Page_30-velocity,Page_30-x_axis_vel,Page_30-y_axis_vel,Page_31-velocity,Page_31-x_axis_vel,Page_31-y_axis_vel,Page_32-velocity,Page_32-x_axis_vel,Page_32-y_axis_vel,Page_33-velocity,Page_33-x_axis_vel,Page_33-y_axis_vel,Page_34-velocity,Page_34-x_axis_vel,Page_34-y_axis_vel,Page_35-velocity,Page_35-x_axis_vel,Page_35-y_axis_vel,Page_36-velocity,Page_36-x_axis_vel,Page_36-y_axis_vel,Page_37-velocity,Page_37-x_axis_vel,Page_37-y_axis_vel,Page_38-velocity,Page_38-x_axis_vel,Page_38-y_axis_vel,Page_39-velocity,Page_39-x_axis_vel,Page_39-y_axis_vel,Page_40-velocity,Page_40-x_axis_vel,Page_40-y_axis_vel,Page_41-velocity,Page_41-x_axis_vel,Page_41-y_axis_vel,Page_42-velocity,Page_42-x_axis_vel,Page_42-y_axis_vel,Page_43-velocity,Page_43-x_axis_vel,Page_43-y_axis_vel,Page_44-velocity,Page_44-x_axis_vel,Page_44-y_axis_vel,Page_45-velocity,Page_45-x_axis_vel,Page_45-y_axis_vel,Page_46-velocity,Page_46-x_axis_vel,Page_46-y_axis_vel,Page_47-velocity,Page_47-x_axis_vel,Page_47-y_axis_vel,Page_48-velocity,Page_48-x_axis_vel,Page_48-y_axis_vel,Page_49-velocity,Page_49-x_axis_vel,Page_49-y_axis_vel,Page_50-velocity,Page_50-x_axis_vel,Page_50-y_axis_vel,Page_51-velocity,Page_51-x_axis_vel,Page_51-y_axis_vel,Page_52-velocity,Page_52-x_axis_vel,Page_52-y_axis_vel,Page_53-velocity,Page_53-x_axis_vel,Page_53-y_axis_vel,Page_54-velocity,Page_54-x_axis_vel,Page_54-y_axis_vel,Page_55-velocity,Page_55-x_axis_vel,Page_55-y_axis_vel,Page_56-velocity,Page_56-x_axis_vel,Page_56-y_axis_vel,Page_57-velocity,Page_57-x_axis_vel,Page_57-y_axis_vel,Page_58-velocity,Page_58-x_axis_vel,Page_58-y_axis_vel,Page_59-velocity,Page_59-x_axis_vel,Page_59-y_axis_vel,Page_60-velocity,Page_60-x_axis_vel,Page_60-y_axis_vel,Page_61-velocity,Page_61-x_axis_vel,Page_61-y_axis_vel,Page_62-velocity,Page_62-x_axis_vel,Page_62-y_axis_vel,Page_63-velocity,Page_63-x_axis_vel,Page_63-y_axis_vel,Page_64-velocity,Page_64-x_axis_vel,Page_64-y_axis_vel,Page_65-velocity,Page_65-x_axis_vel,Page_65-y_axis_vel
0,S5WLXErzUWNQqkDKnRC1j53PD,0.953853,0.862534,0.366115,0.563418,0.325807,0.418193,0.707989,0.32176,0.592622,1.005283,0.878978,0.443896,1.316253,1.210009,0.475594,0.80564,0.668403,0.407135,0.480634,0.245956,0.366928,0.576295,0.391667,0.284343,1.003711,0.886836,0.432331,0.438752,0.400993,0.158495,0.728914,0.620354,0.3328,0.606469,0.31842,0.462411,0.698526,0.309852,0.568182,0.956069,0.827009,0.443182,0.952075,0.836464,0.421278,0.866814,0.773438,0.341371,0.685981,0.413538,0.466443,0.589345,0.303838,0.468968,0.85556,0.734737,0.395034,0.929207,0.8

In [25]:
print_one_metric(df, ['acc', 'x_axis_acc', 'y_axis_acc'], first_page, last_page, show)

,Session,Page_3-acc,Page_3-x_axis_acc,Page_3-y_axis_acc,Page_4-acc,Page_4-x_axis_acc,Page_4-y_axis_acc,Page_5-acc,Page_5-x_axis_acc,Page_5-y_axis_acc,Page_6-acc,Page_6-x_axis_acc,Page_6-y_axis_acc,Page_7-acc,Page_7-x_axis_acc,Page_7-y_axis_acc,Page_8-acc,Page_8-x_axis_acc,Page_8-y_axis_acc,Page_9-acc,Page_9-x_axis_acc,Page_9-y_axis_acc,Page_10-acc,Page_10-x_axis_acc,Page_10-y_axis_acc,Page_11-acc,Page_11-x_axis_acc,Page_11-y_axis_acc,Page_12-acc,Page_12-x_axis_acc,Page_12-y_axis_acc,Page_13-acc,Page_13-x_axis_acc,Page_13-y_axis_acc,Page_14-acc,Page_14-x_axis_acc,Page_14-y_axis_acc,Page_15-acc,Page_15-x_axis_acc,Page_15-y_axis_acc,Page_16-acc,Page_16-x_axis_acc,Page_16-y_axis_acc,Page_17-acc,Page_17-x_axis_acc,Page_17-y_axis_acc,Page_18-acc,Page_18-x_axis_acc,Page_18-y_axis_acc,Page_19-acc,Page_19-x_axis_acc,Page_19-y_axis_acc,Page_20-acc,Page_20-x_axis_acc,Page_20-y_axis_acc,Page_21-acc,Page_21-x_axis_acc,Page_21-y_axis_acc,Page_22-acc,Page_22-x_axis_acc,Page_22-y_axis_acc,Page_23-acc,Page_23-x_axis_acc,Page_23-y_axis_acc,Page_24-acc,Page_24-x_axis_acc,Page_24-y_axis_acc,Page_25-acc,Page_25-x_axis_acc,Page_25-y_axis_acc,Page_26-acc,Page_26-x_axis_acc,Page_26-y_axis_acc,Page_27-acc,Page_27-x_axis_acc,Page_27-y_axis_acc,Page_28-acc,Page_28-x_axis_acc,Page_28-y_axis_acc,Page_29-acc,Page_29-x_axis_acc,Page_29-y_axis_acc,Page_30-acc,Page_30-x_axis_acc,Page_30-y_axis_acc,Page_31-acc,Page_31-x_axis_acc,Page_31-y_axis_acc,Page_32-acc,Page_32-x_axis_acc,Page_32-y_axis_acc,Page_33-acc,Page_33-x_axis_acc,Page_33-y_axis_acc,Page_34-acc,Page_34-x_axis_acc,Page_34-y_axis_acc,Page_35-acc,Page_35-x_axis_acc,Page_35-y_axis_acc,Page_36-acc,Page_36-x_axis_acc,Page_36-y_axis_acc,Page_37-acc,Page_37-x_axis_acc,Page_37-y_axis_acc,Page_38-acc,Page_38-x_axis_acc,Page_38-y_axis_acc,Page_39-acc,Page_39-x_axis_acc,Page_39-y_axis_acc,Page_40-acc,Page_40-x_axis_acc,Page_40-y_axis_acc,Page_41-acc,Page_41-x_axis_acc,Page_41-y_axis_acc,Page_42-acc,Page_42-x_axis_acc,Page_42-y_axis_acc,Page_43-acc,Page_43-x_axis_acc,Page_43-y_axis_acc,Page_44-acc,Page_44-x_axis_acc,Page_44-y_axis_acc,Page_45-acc,Page_45-x_axis_acc,Page_45-y_axis_acc,Page_46-acc,Page_46-x_axis_acc,Page_46-y_axis_acc,Page_47-acc,Page_47-x_axis_acc,Page_47-y_axis_acc,Page_48-acc,Page_48-x_axis_acc,Page_48-y_axis_acc,Page_49-acc,Page_49-x_axis_acc,Page_49-y_axis_acc,Page_50-acc,Page_50-x_axis_acc,Page_50-y_axis_acc,Page_51-acc,Page_51-x_axis_acc,Page_51-y_axis_acc,Page_52-acc,Page_52-x_axis_acc,Page_52-y_axis_acc,Page_53-acc,Page_53-x_axis_acc,Page_53-y_axis_acc,Page_54-acc,Page_54-x_axis_acc,Page_54-y_axis_acc,Page_55-acc,Page_55-x_axis_acc,Page_55-y_axis_acc,Page_56-acc,Page_56-x_axis_acc,Page_56-y_axis_acc,Page_57-acc,Page_57-x_axis_acc,Page_57-y_axis_acc,Page_58-acc,Page_58-x_axis_acc,Page_58-y_axis_acc,Page_59-acc,Page_59-x_axis_acc,Page_59-y_axis_acc,Page_60-acc,Page_60-x_axis_acc,Page_60-y_axis_acc,Page_61-acc,Page_61-x_axis_acc,Page_61-y_axis_acc,Page_62-acc,Page_62-x_axis_acc,Page_62-y_axis_acc,Page_63-acc,Page_63-x_axis_acc,Page_63-y_axis_acc,Page_64-acc,Page_64-x_axis_acc,Page_64-y_axis_acc,Page_65-acc,Page_65-x_axis_acc,Page_65-y_axis_acc
0,S5WLXErzUWNQqkDKnRC1j53PD,0.001169,0.001057,0.000449,0.000688,0.000398,0.000511,0.001111,0.000505,0.00093,0.00124,0.001084,0.000547,0.001647,0.001514,0.000595,0.000878,0.000728,0.000444,0.00047,0.000241,0.000359,0.000233,0.000158,0.000115,0.001258,0.001111,0.000542,0.000371,0.000339,0.000134,0.000667,0.000568,0.000304,0.000724,0.00038,0.000552,0.001076,0.000477,0.000875,0.001242,0.001074,0.000576,0.000958,0.000842,0.000424,0.00088,0.000785,0.000347,0.000921,0.000555,0.000626,0.000822,0.000424,0.000654,0.000966,0.000829,0.000446,0.000965,0.000883,0.000361,0.000736,0.000658,0.000314,0.000278,0.000162,0.000193,0.000996,0.000485,0.000773,0.001564,0.001351,0.00075,0.000707,0.000658,0.000227,0.000826,0.000711,0.000396,0.000682,0.00046,0.000433,0.000582,0.000281,0.000456,0.000578,0.000493,0.000284,0.00081,0.000769,0.000236,0.000708,0.00061,0.00033,0.00051

In [26]:
print_one_metric(df, ['auc_diff'], first_page, last_page, show)

,Session,Page_3-auc_diff,Page_4-auc_diff,Page_5-auc_diff,Page_6-auc_diff,Page_7-auc_diff,Page_8-auc_diff,Page_9-auc_diff,Page_10-auc_diff,Page_11-auc_diff,Page_12-auc_diff,Page_13-auc_diff,Page_14-auc_diff,Page_15-auc_diff,Page_16-auc_diff,Page_17-auc_diff,Page_18-auc_diff,Page_19-auc_diff,Page_20-auc_diff,Page_21-auc_diff,Page_22-auc_diff,Page_23-auc_diff,Page_24-auc_diff,Page_25-auc_diff,Page_26-auc_diff,Page_27-auc_diff,Page_28-auc_diff,Page_29-auc_diff,Page_30-auc_diff,Page_31-auc_diff,Page_32-auc_diff,Page_33-auc_diff,Page_34-auc_diff,Page_35-auc_diff,Page_36-auc_diff,Page_37-auc_diff,Page_38-auc_diff,Page_39-auc_diff,Page_40-auc_diff,Page_41-auc_diff,Page_42-auc_diff,Page_43-auc_diff,Page_44-auc_diff,Page_45-auc_diff,Page_46-auc_diff,Page_47-auc_diff,Page_48-auc_diff,Page_49-auc_diff,Page_50-auc_diff,Page_51-auc_diff,Page_52-auc_diff,Page_53-auc_diff,Page_54-auc_diff,Page_55-auc_diff,Page_56-auc_diff,Page_57-auc_diff,Page_58-auc_diff,Page_59-auc_diff,Page_60-auc_diff,Page_61-auc_diff,Page_62-auc_diff,Page_63-auc_diff,Page_64-auc_diff,Page_65-auc_diff
0,S5WLXErzUWNQqkDKnRC1j53PD,13055.786133,76693.579102,18051.708984,145439.282227,85514.794922,119714.428711,90610.620117,94653.442383,22057.470703,183291.796875,86417.944336,96745.751953,17428.125,70721.191406,293187.597656,70103.24707,21076.171875,79675.341797,94078.198242,311480.200195,67992.407227,99600.219727,20699.926758,15961.010742,334757.446289,100258.447266,108869.384766,74187.158203,15806.323242,298810.327148,105460.620117,81900.585938,106868.920898,29629.101562,327571.728516,80758.154297,85666.259766,11744.165039,69370.092773,322573.388672,129396.899414,70377.978516,14801.660156,74850.219727,331566.210938,106262.255859,19243.286133,14865.307617,68445.19043,105934.350586,18914.575195,90952.22168,44545.166016,305681.030273,27700.341797,113991.796875,107288.671875,15633.911133,269005.59082,106880.200195,3228.295898,64752.026367,28684.057617
1,gsd4FIhsnwaOIMOtoSqX4geVy,50776.6875,161737.125,74979.609375,18163.40625,91599.75,247762.96875,81033.5625,20303.765625,191945.53125,49347.375,166170.46875,82456.6875,285142.6875,113918.578125,13310.34375,40660.640625,176500.5,143743.359375,95373.609375,100892.859375,125908.40625,177008.90625,91884.890625,90454.03125,188641.40625,86667.796875,167027.4375,72289.59375,126787.546875,4028.0625,61795.59375,82462.875,1468.5,113925.28125,127257.796875,277274.765625,25383.703125,94510.96875,163213.875,79516.59375,68149.125,284898.796875,77913.0,63661.125,83259.0,8221.125,180841.03125,183531.046875,67276.171875,98247.703125,153576.328125,156268.40625,88422.984375,69465.515625,171871.21875,78180.09375,47737.078125,82093.171875,107533.078125,61849.734375,95276.671875,83149.6875,8840.90625
2,IKW3ZZ9lcX5Z3vSKEghI73tuI,15924.975,61418.775,100700.325,108159.975,7275.675,200802.525,14259.3,46869.9,71251.125,155944.8,115768.125,207963.525,189095.775,135146.55,164056.2,68292.675,237927.525,195631.425,79669.425,32916.675,56272.425,45261.15,61590.375,100737.45,135559.05,78134.925,1407.45,88086.075,97323.6,45060.675,166881.0,26151.675,76405.725,76648.275,208088.1,96652.875,51591.375,45424.5,192641.625,111869.175,79607.55,181299.525,179765.85,61807.35,42241.65,4536.675,61342.875,95035.05,2858.625,99211.2,212135.55,144171.225,176634.15,192906.45,160448.475,65418.375,16006.65,79438.425,123132.075,195718.875,72373.95,45777.6,56788.875
3,DrTk41pNckxJYIdilpta9qJBT,40341.984375,19835.0625,112420.6875,61317.609375,159502.921875,115265.90625,185116.078125,85483.921875,153266.953125,47007.46875,166400.953125,185605.40625,59293.78125,19663.359375,172795.21875,59009.15625,190925.109375,39246.796875,96936.984375,6723.234375,157666.265625,54977.484375,164251.3125,184707.703125,129615.234375,90078.140625,169859.25,82073.578125,173062.828125,92608.3125,218403.796875,104036.109375,113393.15625,141831.421875,213123.796875,72741.796875,268991.765625,34791.28125,254202.09375,88826.203125,46347.984375,203435.71875,91070.203125,202809.234375,16761.9375,361106.109375,17159

In [27]:
print_one_metric(df, ['clicks'], first_page, last_page, show)

,Session,Page_3-clicks,Page_4-clicks,Page_5-clicks,Page_6-clicks,Page_7-clicks,Page_8-clicks,Page_9-clicks,Page_10-clicks,Page_11-clicks,Page_12-clicks,Page_13-clicks,Page_14-clicks,Page_15-clicks,Page_16-clicks,Page_17-clicks,Page_18-clicks,Page_19-clicks,Page_20-clicks,Page_21-clicks,Page_22-clicks,Page_23-clicks,Page_24-clicks,Page_25-clicks,Page_26-clicks,Page_27-clicks,Page_28-clicks,Page_29-clicks,Page_30-clicks,Page_31-clicks,Page_32-clicks,Page_33-clicks,Page_34-clicks,Page_35-clicks,Page_36-clicks,Page_37-clicks,Page_38-clicks,Page_39-clicks,Page_40-clicks,Page_41-clicks,Page_42-clicks,Page_43-clicks,Page_44-clicks,Page_45-clicks,Page_46-clicks,Page_47-clicks,Page_48-clicks,Page_49-clicks,Page_50-clicks,Page_51-clicks,Page_52-clicks,Page_53-clicks,Page_54-clicks,Page_55-clicks,Page_56-clicks,Page_57-clicks,Page_58-clicks,Page_59-clicks,Page_60-clicks,Page_61-clicks,Page_62-clicks,Page_63-clicks,Page_64-clicks,Page_65-clicks
0,S5WLXErzUWNQqkDKnRC1j53PD,2,2,2,2,2,2,2,4,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1,gsd4FIhsnwaOIMOtoSqX4geVy,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,3,2,2,3,2,2,2,7,2,3,2,2,4,2,2,2,1,2,2,2,2,2,2,2,2,2,2,3,3,3,2
2,IKW3ZZ9lcX5Z3vSKEghI73tuI,3,3,2,4,2,2,3,2,2,2,2,2,2,2,2,2,3,2,2,3,2,2,2,2,2,2,3,2,2,2,2,2,2,3,2,2,3,3,2,4,2,2,3,2,2,5,2,3,1,2,3,2,2,3,2,2,2,2,2,2,2,2,3
3,DrTk41pNckxJYIdilpta9qJBT,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,3,3,3,4,3,2,2,2,3,2,2,2,4,2,2,2,2,3,2,3,2,2,2,2,2,2,2,3,3,2,2,1,2,2,2,2,2,2,2,3,2,2,2,2,3,2
4,jcvlt3renH3FwZuO0vUqmgbKt,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,3,2,2,2,3,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,3
5,PQrxFN8wOy7kHNw0cGORMXIIq,2,2,2,2,2,2,2,2,3,2,3,2,2,2,2,3,3,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,1,2,3,3,2,2,2,2,2,2,2,2,2,2,2
6,k3qdYWdIZkboxvmar40fTzyDW,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,xEoYPtO0Mbwjg5PkET57lc4HB,4,3,4,3,2,3,6,2,2,2,2,2,3,3,2,3,2,2,2,3,2,2,2,2,2,2,3,2,5,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,3,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,4
8,93npiDWQK6VipNjEWLfWafqsr,1,1,1,1,1,1,1,1,2,2,1,1,1,1,2,2,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,2,1,1,1,1,1,2,1,1,1
9,0GNYlUU5mGtxOPKr275ut1hI7,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,0,2


In [28]:
print_one_metric(df, ['x_flips', 'y_flips'], first_page, last_page, show)

,Session,Page_3-x_flips,Page_3-y_flips,Page_4-x_flips,Page_4-y_flips,Page_5-x_flips,Page_5-y_flips,Page_6-x_flips,Page_6-y_flips,Page_7-x_flips,Page_7-y_flips,Page_8-x_flips,Page_8-y_flips,Page_9-x_flips,Page_9-y_flips,Page_10-x_flips,Page_10-y_flips,Page_11-x_flips,Page_11-y_flips,Page_12-x_flips,Page_12-y_flips,Page_13-x_flips,Page_13-y_flips,Page_14-x_flips,Page_14-y_flips,Page_15-x_flips,Page_15-y_flips,Page_16-x_flips,Page_16-y_flips,Page_17-x_flips,Page_17-y_flips,Page_18-x_flips,Page_18-y_flips,Page_19-x_flips,Page_19-y_flips,Page_20-x_flips,Page_20-y_flips,Page_21-x_flips,Page_21-y_flips,Page_22-x_flips,Page_22-y_flips,Page_23-x_flips,Page_23-y_flips,Page_24-x_flips,Page_24-y_flips,Page_25-x_flips,Page_25-y_flips,Page_26-x_flips,Page_26-y_flips,Page_27-x_flips,Page_27-y_flips,Page_28-x_flips,Page_28-y_flips,Page_29-x_flips,Page_29-y_flips,Page_30-x_flips,Page_30-y_flips,Page_31-x_flips,Page_31-y_flips,Page_32-x_flips,Page_32-y_flips,Page_33-x_flips,Page_33-y_flips,Page_34-x_flips,Page_34-y_flips,Page_35-x_flips,Page_35-y_flips,Page_36-x_flips,Page_36-y_flips,Page_37-x_flips,Page_37-y_flips,Page_38-x_flips,Page_38-y_flips,Page_39-x_flips,Page_39-y_flips,Page_40-x_flips,Page_40-y_flips,Page_41-x_flips,Page_41-y_flips,Page_42-x_flips,Page_42-y_flips,Page_43-x_flips,Page_43-y_flips,Page_44-x_flips,Page_44-y_flips,Page_45-x_flips,Page_45-y_flips,Page_46-x_flips,Page_46-y_flips,Page_47-x_flips,Page_47-y_flips,Page_48-x_flips,Page_48-y_flips,Page_49-x_flips,Page_49-y_flips,Page_50-x_flips,Page_50-y_flips,Page_51-x_flips,Page_51-y_flips,Page_52-x_flips,Page_52-y_flips,Page_53-x_flips,Page_53-y_flips,Page_54-x_flips,Page_54-y_flips,Page_55-x_flips,Page_55-y_flips,Page_56-x_flips,Page_56-y_flips,Page_57-x_flips,Page_57-y_flips,Page_58-x_flips,Page_58-y_flips,Page_59-x_flips,Page_59-y_flips,Page_60-x_flips,Page_60-y_flips,Page_61-x_flips,Page_61-y_flips,Page_62-x_flips,Page_62-y_flips,Page_63-x_flips,Page_63-y_flips,Page_64-x_flips,Page_64-y_flips,Page_65-x_flips,Page_65-y_flips
0,S5WLXErzUWNQqkDKnRC1j53PD,1,1,2,1,0,1,1,1,0,1,1,3,1,1,7,8,1,1,0,1,2,2,2,1,2,1,1,1,3,2,2,2,1,1,3,1,1,1,1,2,2,2,1,2,0,1,1,1,1,1,2,2,2,2,3,2,1,1,1,2,2,2,2,2,0,2,1,1,0,2,1,1,2,2,0,1,2,2,0,1,2,1,2,2,2,1,2,2,1,2,2,2,1,1,2,1,0,0,2,2,1,1,0,1,2,2,1,2,2,2,1,1,0,2,1,1,1,1,2,2,1,1,2,2,2,2
1,gsd4FIhsnwaOIMOtoSqX4geVy,8,8,2,1,2,4,2,3,2,7,2,2,1,1,2,1,1,3,4,2,2,2,2,2,1,2,3,2,2,1,2,2,2,1,2,2,1,1,1,1,1,1,2,1,1,1,2,1,1,1,1,3,4,5,2,4,1,1,2,2,1,1,2,4,6,7,1,2,1,1,3,4,1,3,2,2,4,2,15,16,2,2,1,2,1,1,3,1,2,4,5,4,1,1,1,2,0,1,1,1,2,3,2,4,1,1,3,1,4,4,4,2,4,2,0,1,1,2,6,6,1,1,2,2,1,2
2,IKW3ZZ9lcX5Z3vSKEghI73tuI,5,6,7,8,11,9,6,8,3,3,2,2,3,3,13,12,4,1,3,3,3,7,2,1,1,1,3,2,2,2,2,1,2,5,1,1,2,1,5,4,2,1,4,2,5,4,3,2,2,1,3,3,1,3,5,3,3,1,4,2,2,4,2,1,1,1,3,3,3,1,4,1,6,4,7,5,1,1,7,5,3,1,4,1,3,1,5,5,4,4,3,5,5,4,7,5,4,1,4,6,3,2,2,2,3,3,5,4,6,3,3,1,4,3,2,1,2,2,6,3,3,3,3,1,6,4
3,DrTk41pNckxJYIdilpta9qJBT,6,8,4,1,6,6,6,6,4,7,4,6,6,3,2,2,6,7,2,5,3,4,5,8,5,2,3,4,2,3,8,3,1,1,9,8,3,4,5,6,5,5,10,14,4,4,4,5,8,5,10,18,2,7,5,6,6,6,8,8,12,8,4,4,2,4,2,7,1,6,3,4,6,9,3,1,12,10,9,7,7,6,11,8,2,7,4,3,7,7,4,9,3,5,7,6,1,1,4,6,8,7,1,1,2,1,2,3,3,2,2,3,8,10,2,2,3,2,3,1,3,4,5,4,4,2
4,jcvlt3renH3FwZuO0vUqmgbKt,15,11,15,17,3,2,4,4,3,1,1,1,5,4,2,1,4,4,2,2,3,4,3,1,2,2,4,8,2,2,2,1,5,6,3,4,8,8,2,1,2,2,3,1,3,1,5,5,5,1,2,1,3,4,6,1,2,1,4,4,4,1,3,1,5,2,3,1,2,2,1,1,2,2,3,1,4,4,4,1,2,1,1,1,5,5,2,2,4,4,4,1,2,1,5,3,3,3,4,1,1,2,3,3,4,4,5,3,2,4,5,7,3,5,3,3,1,3,3,2,4,1,2,1,3,1
5,PQrxFN8wOy7kHNw0cGORMXIIq,1,3,2,2,1,3,1,1,1,4,2,1,1,1,3,1,6,6,1,2,5,3,5,4,1,2,2,1,3,1,1,2,1,4,4,2,1,1,4,2,1,2,4,3,2,4,3,5,2,1,1,1,2,2,4,1,1,3,2,1,1,1,2,1,4,3,3,1,1,1,3,2,3,2,2,2,2,2,2,2,2,2,1,2,2,2,3,2,3,2,4,1,4,2,6,4,1,1,3,2,4,6,8,7,1,2,2,2,3,8,1,1,4,2,1,2,2,1,2,3,1,1,2,3,1,3
6,k3qdYWdIZkboxvmar40fTzyDW,9,8,1,2,2,2,4,5,4,5,2,2,2,3,2,1,5,6,2,2,2,1,2,7,3,4,1,2,2,1,1,2,5,5,1,3,3,2,2,1,3,1,4,2,3,4,2,3,4,2,2,5,1,1,2,2,8,5,5,10,2,1,2,4,2,1,1,2,1,2,3,4,2,2,4,5,3,3,3,1,3,3,2,1,4,2,3,3,4,5,5,1,2,3,4,10,0,0,6,6,4,6,4,5,3,1,3,2,4,4,2,1,3,7,1,2,2,1,3,4,2,2,2,2,6,5
7,xEoYPtO

In [29]:
print_one_metric(df, ['visits', 'scrolling'], first_page, last_page, show)

,Session,Page_3-visits,Page_3-scrolling,Page_4-visits,Page_4-scrolling,Page_5-visits,Page_5-scrolling,Page_6-visits,Page_6-scrolling,Page_7-visits,Page_7-scrolling,Page_8-visits,Page_8-scrolling,Page_9-visits,Page_9-scrolling,Page_10-visits,Page_10-scrolling,Page_11-visits,Page_11-scrolling,Page_12-visits,Page_12-scrolling,Page_13-visits,Page_13-scrolling,Page_14-visits,Page_14-scrolling,Page_15-visits,Page_15-scrolling,Page_16-visits,Page_16-scrolling,Page_17-visits,Page_17-scrolling,Page_18-visits,Page_18-scrolling,Page_19-visits,Page_19-scrolling,Page_20-visits,Page_20-scrolling,Page_21-visits,Page_21-scrolling,Page_22-visits,Page_22-scrolling,Page_23-visits,Page_23-scrolling,Page_24-visits,Page_24-scrolling,Page_25-visits,Page_25-scrolling,Page_26-visits,Page_26-scrolling,Page_27-visits,Page_27-scrolling,Page_28-visits,Page_28-scrolling,Page_29-visits,Page_29-scrolling,Page_30-visits,Page_30-scrolling,Page_31-visits,Page_31-scrolling,Page_32-visits,Page_32-scrolling,Page_33-visits,Page_33-scrolling,Page_34-visits,Page_34-scrolling,Page_35-visits,Page_35-scrolling,Page_36-visits,Page_36-scrolling,Page_37-visits,Page_37-scrolling,Page_38-visits,Page_38-scrolling,Page_39-visits,Page_39-scrolling,Page_40-visits,Page_40-scrolling,Page_41-visits,Page_41-scrolling,Page_42-visits,Page_42-scrolling,Page_43-visits,Page_43-scrolling,Page_44-visits,Page_44-scrolling,Page_45-visits,Page_45-scrolling,Page_46-visits,Page_46-scrolling,Page_47-visits,Page_47-scrolling,Page_48-visits,Page_48-scrolling,Page_49-visits,Page_49-scrolling,Page_50-visits,Page_50-scrolling,Page_51-visits,Page_51-scrolling,Page_52-visits,Page_52-scrolling,Page_53-visits,Page_53-scrolling,Page_54-visits,Page_54-scrolling,Page_55-visits,Page_55-scrolling,Page_56-visits,Page_56-scrolling,Page_57-visits,Page_57-scrolling,Page_58-visits,Page_58-scrolling,Page_59-visits,Page_59-scrolling,Page_60-visits,Page_60-scrolling,Page_61-visits,Page_61-scrolling,Page_62-visits,Page_62-scrolling,Page_63-visits,Page_63-scrolling,Page_64-visits,Page_64-scrolling,Page_65-visits,Page_65-scrolling
0,S5WLXErzUWNQqkDKnRC1j53PD,1,False,1,False,1,False,1,False,1,False,1,False,1,False,2,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False
1,gsd4FIhsnwaOIMOtoSqX4geVy,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False
2,IKW3ZZ9lcX5Z3vSKEghI73tuI,1,False,1,False,1,False,1,False,1,False,1,False,1,False,2,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False
3,DrTk41pNckxJYIdilpta9qJBT,2,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,False,1,Fals

In [30]:
print_one_metric(df, ['init_time', 'react_time'], first_page, last_page, show)

,Session,Page_3-init_time,Page_3-react_time,Page_4-init_time,Page_4-react_time,Page_5-init_time,Page_5-react_time,Page_6-init_time,Page_6-react_time,Page_7-init_time,Page_7-react_time,Page_8-init_time,Page_8-react_time,Page_9-init_time,Page_9-react_time,Page_10-init_time,Page_10-react_time,Page_11-init_time,Page_11-react_time,Page_12-init_time,Page_12-react_time,Page_13-init_time,Page_13-react_time,Page_14-init_time,Page_14-react_time,Page_15-init_time,Page_15-react_time,Page_16-init_time,Page_16-react_time,Page_17-init_time,Page_17-react_time,Page_18-init_time,Page_18-react_time,Page_19-init_time,Page_19-react_time,Page_20-init_time,Page_20-react_time,Page_21-init_time,Page_21-react_time,Page_22-init_time,Page_22-react_time,Page_23-init_time,Page_23-react_time,Page_24-init_time,Page_24-react_time,Page_25-init_time,Page_25-react_time,Page_26-init_time,Page_26-react_time,Page_27-init_time,Page_27-react_time,Page_28-init_time,Page_28-react_time,Page_29-init_time,Page_29-react_time,Page_30-init_time,Page_30-react_time,Page_31-init_time,Page_31-react_time,Page_32-init_time,Page_32-react_time,Page_33-init_time,Page_33-react_time,Page_34-init_time,Page_34-react_time,Page_35-init_time,Page_35-react_time,Page_36-init_time,Page_36-react_time,Page_37-init_time,Page_37-react_time,Page_38-init_time,Page_38-react_time,Page_39-init_time,Page_39-react_time,Page_40-init_time,Page_40-react_time,Page_41-init_time,Page_41-react_time,Page_42-init_time,Page_42-react_time,Page_43-init_time,Page_43-react_time,Page_44-init_time,Page_44-react_time,Page_45-init_time,Page_45-react_time,Page_46-init_time,Page_46-react_time,Page_47-init_time,Page_47-react_time,Page_48-init_time,Page_48-react_time,Page_49-init_time,Page_49-react_time,Page_50-init_time,Page_50-react_time,Page_51-init_time,Page_51-react_time,Page_52-init_time,Page_52-react_time,Page_53-init_time,Page_53-react_time,Page_54-init_time,Page_54-react_time,Page_55-init_time,Page_55-react_time,Page_56-init_time,Page_56-react_time,Page_57-init_time,Page_57-react_time,Page_58-init_time,Page_58-react_time,Page_59-init_time,Page_59-react_time,Page_60-init_time,Page_60-react_time,Page_61-init_time,Page_61-react_time,Page_62-init_time,Page_62-react_time,Page_63-init_time,Page_63-react_time,Page_64-init_time,Page_64-react_time,Page_65-init_time,Page_65-react_time
0,S5WLXErzUWNQqkDKnRC1j53PD,782,763,232,341,235,194,126,241,279,245,110,218,106,214,144,1795,283,218,120,234,130,394,124,244,305,224,254,255,161,330,120,273,263,240,243,262,131,246,123,299,126,321,143,412,310,248,290,234,126,280,142,337,149,267,123,305,136,270,134,338,118,378,140,271,143,324,271,257,121,307,320,786,154,323,285,260,119,303,132,253,145,398,141,298,285,238,151,289,126,302,126,272,252,247,274,207,115,204,405,914,322,261,134,252,140,305,127,316,147,341,142,244,130,340,278,240,143,363,143,433,389,249,131,423,140,368
1,gsd4FIhsnwaOIMOtoSqX4geVy,261,7861,470,2520,281,3316,2264,3262,417,3670,785,4543,1834,2263,282,2641,149,4571,140,4826,140,1943,2160,2870,1626,2441,2262,2892,3455,3925,331,4016,1190,2221,1522,2016,2564,3202,2995,3480,2252,2697,708,4418,555,2571,4493,5129,2257,3543,204,3259,1710,2779,1531,2937,4134,5020,150,5158,2295,2974,157,1957,1176,3382,138,4149,2023,3253,1668,2834,2024,3949,1803,2440,4158,4659,2651,10186,4888,5892,1752,2485,3373,4482,432,4773,991,5494,320,3418,137,2317,384,2582,7274,0,1311,2157,714,2553,302,1722,957,1558,793,2044,2689,4374,548,2950,1735,3326,2900,3835,288,2773,139,8071,1196,1754,629,2576,3782,4213
2,IKW3ZZ9lcX5Z3vSKEghI73tuI,2221,9238,1865,6108,690,10176,493,5247,148,5170,1924,2697,1817,3283,1107,20310,839,6476,2040,5001,1266,5824,901,2148,931,2205,1080,3700,1100,3284,2329,3177,1034,2623,2089,2663,1693,2413,4852,12989,1937,2777,634,3019,948,4059,157,3228,1618,2443,1966,4195,11843,13881,1999,6080,4618,5806,1100,2534,322,4542,1771,2827,925,2463,2309,6008,2007,3194,681,2819,445,3543,1437,4857,1914,2613,459,5614,2929,4124,598,2002,463,3567,2400,5356,576,3548,1668,4872,706,2776,1051,7256,136,0,915,6703,640,

Saving generated dataframe into JSON.

In [31]:
result = df.to_json(orient="records")
with open(SESSION_METRICS_FROM_BAKED_JSON, 'w') as outfile:
    outfile.write(result)

Checking some null values

In [32]:
# 3, 51
# 52, 66

df_itl = print_one_metric(df, ['real_ideal_trajectory_diff'], 3, 51, 150)
df_itl

,Session,Page_3-real_ideal_trajectory_diff,Page_4-real_ideal_trajectory_diff,Page_5-real_ideal_trajectory_diff,Page_6-real_ideal_trajectory_diff,Page_7-real_ideal_trajectory_diff,Page_8-real_ideal_trajectory_diff,Page_9-real_ideal_trajectory_diff,Page_10-real_ideal_trajectory_diff,Page_11-real_ideal_trajectory_diff,Page_12-real_ideal_trajectory_diff,Page_13-real_ideal_trajectory_diff,Page_14-real_ideal_trajectory_diff,Page_15-real_ideal_trajectory_diff,Page_16-real_ideal_trajectory_diff,Page_17-real_ideal_trajectory_diff,Page_18-real_ideal_trajectory_diff,Page_19-real_ideal_trajectory_diff,Page_20-real_ideal_trajectory_diff,Page_21-real_ideal_trajectory_diff,Page_22-real_ideal_trajectory_diff,Page_23-real_ideal_trajectory_diff,Page_24-real_ideal_trajectory_diff,Page_25-real_ideal_trajectory_diff,Page_26-real_ideal_trajectory_diff,Page_27-real_ideal_trajectory_diff,Page_28-real_ideal_trajectory_diff,Page_29-real_ideal_trajectory_diff,Page_30-real_ideal_trajectory_diff,Page_31-real_ideal_trajectory_diff,Page_32-real_ideal_trajectory_diff,Page_33-real_ideal_trajectory_diff,Page_34-real_ideal_trajectory_diff,Page_35-real_ideal_trajectory_diff,Page_36-real_ideal_trajectory_diff,Page_37-real_ideal_trajectory_diff,Page_38-real_ideal_trajectory_diff,Page_39-real_ideal_trajectory_diff,Page_40-real_ideal_trajectory_diff,Page_41-real_ideal_trajectory_diff,Page_42-real_ideal_trajectory_diff,Page_43-real_ideal_trajectory_diff,Page_44-real_ideal_trajectory_diff,Page_45-real_ideal_trajectory_diff,Page_46-real_ideal_trajectory_diff,Page_47-real_ideal_trajectory_diff,Page_48-real_ideal_trajectory_diff,Page_49-real_ideal_trajectory_diff,Page_50-real_ideal_trajectory_diff
0,S5WLXErzUWNQqkDKnRC1j53PD,201.157349,162.537787,231.561288,565.961453,473.933526,243.197912,196.464719,1213.576216,512.544937,10.615316,298.498805,199.669431,230.816882,451.926073,457.042004,341.52884,159.322119,246.349149,473.060209,368.763011,447.65953,216.676798,246.78822,423.312152,455.874743,273.802878,125.393233,271.424162,288.679764,322.007456,246.609753,89.304198,292.512951,422.286797,437.94058,258.463442,142.891247,151.949717,458.646737,406.996677,231.056749,173.766954,205.778556,450.656734,385.434851,285.7151,154.41745,216.092669
1,gsd4FIhsnwaOIMOtoSqX4geVy,3434.874661,456.372879,241.107925,65.496732,177.745514,224.872574,6.391991,68.748172,53.420592,200.828553,163.223266,256.361039,207.645944,177.273912,12.328605,176.83482,71.854629,183.38172,21.710733,12.693101,33.331936,100.246322,40.671436,144.678753,34.810899,31.215215,629.777074,445.836243,4.397713,229.455196,10.536347,51.777927,795.10874,28.820438,19.367741,703.520194,280.019073,205.549844,104.309532,2675.316893,32.530279,260.402414,23.855748,239.976474,476.74278,719.802771,27.918245,45.614227
2,IKW3ZZ9lcX5Z3vSKEghI73tuI,2049.72216,973.352537,2422.172163,866.520438,306.312329,138.889864,807.019017,1670.440423,101.130058,24.042151,1347.234631,18.02836,32.886792,862.181004,247.251555,7.285548,911.366813,29.158897,12.744692,546.899529,557.048568,355.274163,512.844872,57.584374,373.636991,875.699523,608.982228,1156.761966,287.024089,488.783042,61.082179,707.566687,46.528533,1358.482776,228.819185,267.780127,476.362955,1216.247688,49.198181,1142.140432,165.549557,170.26809,73.396378,712.357445,754.351756,1177.780845,607.309163,2016.610684
3,DrTk41pNckxJYIdilpta9qJBT,3133.399076,973.822734,1875.928239,1516.669597,935.652708,457.979146,374.063026,228.395055,1666.855533,955.170933,514.136359,1317.525503,637.499775,301.007279,244.788693,86.781538,28.804973,1007.405442,573.958647,604.211761,653.710129,1341.128582,246.636175,626.102646,728.050974,1110.579454,1070.624594,841.800003,1471.460946,1760.317106,1166.230336,317.582236,403.795429,652.336402,150.142931,725.470807,3901.74873,944.337395,2066.796405,1800.228116,2863.409718,2644.881372,1338.470812,234.947409,753.422176,1975.562782,373.096783,476.247842
4,jcvlt3renH3FwZuO0vUqmgbKt,1973.886959,2556.019802,269.882084,519.816077,157.749281,53.708878,1818.056722,11.469413

In [33]:
nulls = df_itl.isnull().sum(axis=1)
count = 0
for x in nulls:
    if x == max(nulls):
        count += 1
print(max(nulls), count)

3 3
